In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from scipy.spatial import distance
# from scipy.sparse.csgraph import minimum_spanning_tree
from scipy import stats
import math

from sklearn.neighbors import NearestNeighbors

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from modules.dp_func import get_bounds_dp as get_bounds_dp

from modules.tight_knn_func import get_tight_bounds_knn


In [3]:
# Set the mean and covariance matrix for each Gaussian distribution


mean1 = [0, 0, 0 , 0 , 0, 0, 0 , 0]
# covariance1 = np.ones((3,3))
covariance1 = np.identity(8)

mean2 = [2.56, 0, 0 , 0, 0, 0,0, 0 ]

# covariance2 = np.ones((3,3))
covariance2= np.identity(8)

#choose the number of samples per group. 
samples = 200

n0 = samples
n1 = samples

# p, q = .5, .5

<!-- \begin{align*}
p_k(x)  =\frac{k}{n} \frac{1}{ \frac{\pi^{p/2}}{\Gamma(p/2+1)}  \|x-x_k \|}.
\end{align*} -->

\begin{align*}
p_k(x)  =\frac{k}{n} \frac{1}{ \frac{\pi^{p/2}}{\Gamma(p/2+1)}  \|x-x_k \|^p}.
\end{align*}

see here 
https://scikit-learn.org/stable/modules/neighbors.html#nei

In [4]:

def knn_density_calc(distances_matrix, k, p, n): # p is the dimension 
    vec = np.zeros(len(distances_matrix))
        
    gam_p = math.gamma( p /2 + 1)
    pi_power = (math.pi)**(p/ 2)
    vol_mult = pi_power / gam_p

    for i, dist_lst in enumerate(distances_matrix):
        dist = dist_lst[k-1]
        vol=  (dist**p) * vol_mult
        vec[i] =  k /  ( n * vol  )
    
    return vec
    

The cell below and the 2 below that work identically

In [5]:
# def kkn_density_bounds(density1, density2):
#     a = 50
#     gl_half = 1/a * np.log((1 + np.exp(-a)) / (np.exp(-a*1/2) + np.exp(-a*(1/2))))

#     # fx calculation
#     fx = 0.5 * (density1 + density2)  
#     px = density1 / (density1 + density2)

#     # integration
#     glx = 1/a * np.log((1 + np.exp(-a)) / (np.exp(-a*px) + np.exp(-a*(1-px)))) * fx 
    
#     gux = glx + (1 - 2 * gl_half) * 0.5 * np.sin(np.pi * px)* fx 
    
#     return np.mean(glx), np.mean(gux), px


In [6]:
def g_L(p, alpha):
    return 1/alpha * np.log(np.cosh(alpha/2) / np.cosh(alpha * (p - 1/2)))
#     return 1/alpha * np.log((1 + np.exp(-alpha)) / (np.exp(-alpha * p) + np.exp(-alpha * (1 - p))))


def g_C(p):
    return ( 1/2 * np.sin( np.pi * p ) )

def g_U(p, alpha,  g_L, g_C):
    return g_L(p, alpha) + (1 - 2 * g_L(0.5, alpha)) * g_C(p)

In [7]:

def kkn_density_bounds(density1, density2):
    # fx calculation
    
    fx = 0.5 * (density1 + density2)

    px = density1 / (density1 + density2)
    

    alpha = 50
    
    glx = np.mean( g_L(px, alpha)    )
    gux = np.mean( g_U(px, alpha, g_L, g_C)  )
    
    return glx, gux, px

In [8]:

bounds_l = []
bounds_u = []

knn_l = []
knn_u = []

MC_num = 200

k = 10

for i in range(MC_num):
    # Generate random samples from the multivariate normal distributions
    data1 = np.random.multivariate_normal(mean1, covariance1, n0)
    data2 = np.random.multivariate_normal(mean2, covariance2, n1)


    l, u = get_tight_bounds_knn(data1, data2)
    
    knn_l.append(l)
    knn_u.append(u)
    
    
    lower, upper = get_bounds_dp(data1, data2)

    bounds_l.append(lower)
    bounds_u.append(upper)
    

    


In [9]:
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111)

# x_val = np.linspace(1, n0 +n1, n0+n1)
# ax.plot(x_val,density1)

# a = np.mean(density1[:n0])
# b =  np.mean(density1[n0:])
# print( a, b, (a - b)/a)

In [10]:
print(stats.describe(knn_l), "\n")

print(stats.describe(knn_u))

DescribeResult(nobs=200, minmax=(0.11570189252367126, 0.19457942603366254), mean=0.1563159775733549, variance=0.00016418617274881943, skewness=-0.00990970456081027, kurtosis=0.23331414327703692) 

DescribeResult(nobs=200, minmax=(0.12013109622212471, 0.2017364989986315), mean=0.16216926879122626, variance=0.00017569244124778475, skewness=-0.011451768803776047, kurtosis=0.2341530148015436)


In [11]:
print(stats.describe(bounds_l))
print(stats.describe(bounds_u))

DescribeResult(nobs=200, minmax=(0.06267860788660246, 0.14468323991120263), mean=0.09808769016336381, variance=0.00018617608219070672, skewness=0.1800403404337057, kurtosis=0.1745599711620387)
DescribeResult(nobs=200, minmax=(0.1175, 0.2475), mean=0.1765625, variance=0.00047854506909547737, skewness=0.0720566463102384, kurtosis=0.06492360097246719)


In [12]:
dx = 0.001
x = np.arange(-5, 10 + dx, dx)

f0 = np.exp(-0.5 * (x - 0)**2) / np.sqrt(2 * np.pi)
f1 = np.exp(-0.5 * (x - 2.56)**2) / np.sqrt(2 * np.pi)

fmin = np.minimum(f0, f1)

BER = 0.5 * np.sum(fmin * dx)

print("BER 2d:", BER)

BER 2d: 0.10027258671145799


In [13]:
# BER_lst = np.ones(MC_num) * BER


# mse_l = ((BER_lst - bounds_l)**2).mean(axis=0)

# mse_u =  ((BER_lst - bounds_u)**2).mean(axis=0)

# print(mse_l, mse_u)

In [14]:
sum(px)

NameError: name 'px' is not defined

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111)

x_val = np.linspace(1, n0 +n1,n0+n1)
ax.plot(x_val, px)

plt.show()